In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sql-murder-mystery-database/sql-murder-mystery.db


In [4]:
# import some useful libraries
import sqlite3 as sql
import numpy as np
import pandas as np

In [18]:
# Connecting to the database
conn = sql.connect('../input/sql-murder-mystery-database/sql-murder-mystery.db')

In [19]:
# Creating a list of all tables from SQL query
query1 = "SELECT name FROM sqlite_master WHERE type = 'table'"
table_names = list(pd.read_sql_query(query1, conn).values.flatten())[:-1]
table_names

['crime_scene_report',
 'drivers_license',
 'person',
 'facebook_event_checkin',
 'interview',
 'get_fit_now_member',
 'get_fit_now_check_in',
 'income']

In [22]:
# Creating Dataframes from SQL query
for name in table_names:
    query = f"select * from {name}"
    globals()[f"{name}"]  = pd.read_sql(query, conn)
    print(f"Dataframe '{name}' created.")

Dataframe 'crime_scene_report' created.
Dataframe 'drivers_license' created.
Dataframe 'person' created.
Dataframe 'facebook_event_checkin' created.
Dataframe 'interview' created.
Dataframe 'get_fit_now_member' created.
Dataframe 'get_fit_now_check_in' created.
Dataframe 'income' created.


In [49]:
# START OF INVESTIGATION
# 1. Let's investigate the main table "Crime Scene Report"

crime_scene_report.head()

,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


In [50]:
# 2. I spotted, that this report includes all crimes. Let's filter it with our city name as 'SQL City' and type of crime as 'murder'

crime_scene_report[(crime_scene_report["city"]=="SQL City") & (crime_scene_report["type"]=="murder")]

,date,type,description,city
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
1227,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [61]:
# 3. I don't see the while description, so let's see the full length of crime description

for i in range(len(crime_scene_report["description"][(crime_scene_report["city"]=="SQL City") & (crime_scene_report["type"]=="murder")])):
    print(f"Description #{i} is as follows: '{crime_scene_report['description'][(crime_scene_report['city']=='SQL City') & (crime_scene_report['type']=='murder')].iloc[i]}'.")

Description #0 is as follows: 'REDACTED REDACTED REDACTED'.
Description #1 is as follows: 'Someone killed the guard! He took an arrow to the knee!'.
Description #2 is as follows: 'Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".'.


In [138]:
# 4.1 Let's focus on the description #2, which says, that first witness lives at the last house on "Northwestern Dr" Let's find him!

witness_1 = person[(person['address_street_name']=='Northwestern Dr')&(person['address_number']== max(person['address_number'][person['address_street_name']=='Northwestern Dr']))]
witness_1

,id,name,license_id,address_number,address_street_name,ssn
499,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [134]:
# 4.2 Let's focus now on the second witness. The second witness, named Annabel, lives somewhere on "Franklin Ave". Let's find her!

witness_2 = person[(person['address_street_name']=='Franklin Ave')&(person['name'].str.contains("Annabel", case=False, na=False))]
witness_2

,id,name,license_id,address_number,address_street_name,ssn
665,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [149]:
# 5.1 Let's see what our witness_1 said on interview.

interview['transcript'][interview['person_id']==witness_1['id'].iloc[0]].iloc[0]

'I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".'

In [153]:
# 5.2 Let's see what our witness_2 said on interview.

interview['transcript'][interview['person_id']==witness_2['id'].iloc[0]].iloc[0]

'I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.'

In [31]:
drivers_license.head()
# person.head()
# facebook_event_checkin.head()
# interview.head()
# get_fit_now_member.head()
# get_fit_now_check_in.head()
# income.head()

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,100280,72,57,brown,red,male,P24L4U,Acura,MDX
1,100460,63,72,brown,brown,female,XF02T6,Cadillac,SRX
2,101029,62,74,green,green,female,VKY5KR,Scion,xB
3,101198,43,54,amber,brown,female,Y5NZ08,Nissan,Rogue
4,101255,18,79,blue,grey,female,5162Z1,Lexus,GS
